In [ ]:
#Source: https://colab.research.google.com/drive/1U5yTGuuc-sbJ7rN64MVC33JQ858SHvk2?usp=sharing
#Explanation: https://www.youtube.com/watch?v=bfLm584Cz-M&list=PLKYEe2WisBTHOxrEG_COAgJTWnceouhRm&index=3&ab_channel=GregHogg
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d fedesoriano/heart-failure-prediction
!unzip heart-failure-prediction.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open heart-failure-prediction.zip, heart-failure-prediction.zip.zip or heart-failure-prediction.zip.ZIP.


In [ ]:
import pandas as pd

heart = pd.read_csv('heart.csv')
heart

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
import plotly.express as px

px.scatter(heart, x='MaxHR', y='HeartDisease', color='HeartDisease')

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train = heart['MaxHR'].to_numpy().reshape((len(heart['MaxHR']), 1))
y_train = heart['HeartDisease'].to_numpy()

X_train.shape, y_train.shape

((918, 1), (918,))

In [ ]:
lr = LogisticRegression().fit(X_train, y_train)
lr.predict(X_train).shape

(918,)

In [ ]:
lr.predict_proba(X_train).shape

(918, 2)

In [ ]:
from copy import deepcopy

predictions_df = deepcopy(heart[['MaxHR', 'HeartDisease']])
predictions_df['Sk Learn Probability'] = lr.predict_proba(X_train)[:, 1]
predictions_df['SK Learn Prediction'] = (lr.predict_proba(X_train)[:, 1] > 0.5).astype(int)
predictions_df

,MaxHR,HeartDisease,Sk Learn Probability,SK Learn Prediction
0,172,0,0.260939,0
1,156,1,0.389737,0
2,98,0,0.845538,1
3,108,1,0.790775,1
4,122,0,0.692325,1
...,...,...,...,...
913,132,1,0.608400,1
914,141,1,0.526777,1
915,115,1,0.744655,1
916,174,1,0.246907,0


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['HeartDisease'],
                         mode='markers', name='Observed', marker={'color':'blue'}))

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['Sk Learn Probability'],
                         mode='markers', name='Model Predictions', marker={'color':'green'}))

fig.update_layout(
    title="Heart Disease vs Max Heart Rate",
    xaxis_title="Max Heart Rate",
    yaxis_title="Heart Disease"
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['SK Learn Prediction'],
                         mode='markers', name='Model Predictions', marker={'color':'blue'}))

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['Sk Learn Probability'],
                         mode='markers', name='Model Probabilities', marker={'color':'green'}))

fig.update_layout(
    title="Heart Disease vs Max Heart Rate",
    xaxis_title="Max Heart Rate",
    yaxis_title="Heart Disease"
)

fig.show()

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions_df['SK Learn Prediction'], y_train)

0.6710239651416122

In [ ]:
lr.intercept_, lr.coef_

(array([5.33015391]), array([[-0.03704214]]))

In [ ]:
heart.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
import numpy as np

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

sigmoid(0)

0.5

In [ ]:
test_range = np.arange(-50, 50)

fig = go.Figure()
fig.add_trace(go.Scatter(x=test_range, y=sigmoid(test_range)))

In [ ]:
# Assuming Pr(HeartDisease[i]) = Sigmoid(alpha + (beta * MaxHR[i])) + error
# Generally: Pr(y[i]) = Sigmoid(alpha + (beta * x[i])) + error
# Generally: Pr(y[i]) = Sigmoid(alpha_hat + (beta_hat * x[i]))

def get_probabilities(model, x):
  '''
  Obtain the predictions for the given model and inputs.

  model: {'alpha_hat':Number, 'beta_hat':Number}
  x: np.array of Floats with shape (n,) of inputs
  
  Returns: np.array of 0 <= Floats <= 1 with shape (n,),
           representing probabilities
  '''
  alpha_hat = model['alpha_hat']
  beta_hat = model['beta_hat']

  return sigmoid(alpha_hat + beta_hat*x)

In [ ]:
X_train.flatten().shape

(918,)

In [ ]:
test_model = dict(alpha_hat=0, beta_hat=0)
predictions_df['Our Model Heart Disease Probabilities'] = get_probabilities(test_model, X_train.flatten())
predictions_df

,MaxHR,HeartDisease,Sk Learn Probability,SK Learn Prediction,Our Model Heart Disease Probabilities
0,172,0,0.260939,0,0.5
1,156,1,0.389737,0,0.5
2,98,0,0.845538,1,0.5
3,108,1,0.790775,1,0.5
4,122,0,0.692325,1,0.5
...,...,...,...,...,...
913,132,1,0.608400,1,0.5
914,141,1,0.526777,1,0.5
915,115,1,0.744655,1,0.5
916,174,1,0.246907,0,0.5


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['HeartDisease'],
                         mode='markers', name='Observed', marker=dict(color='orange')))

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['Sk Learn Probability'],
                         mode='markers', name='SK Learn Probabilities', marker=dict(color='blue')))

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['Our Model Heart Disease Probabilities'],
                         mode='markers', name='Our Model Heart Disease Probabilities', marker=dict(color='red')))

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=[.5]*len(predictions_df), 
                         name='Threshold', marker={'color':'purple'}))

fig.update_layout(
    title="Heart Disease vs Max Heart Rate",
    xaxis_title="Max Heart Rate",
    yaxis_title="Heart Disease"
)

fig.show()

In [ ]:
accuracy_score(predictions_df['Our Model Heart Disease Probabilities']>.5, y_train)

0.4466230936819172

In [ ]:
from sklearn.metrics import mean_squared_error

def function_to_minimize(params, *args):
  alpha_hat = params[0]
  beta_hat = params[1]
  model = {'alpha_hat':alpha_hat, 'beta_hat':beta_hat}
  x = args[0]
  y = args[1]
  prediction_probs = get_probabilities(model, x)

  return mean_squared_error(prediction_probs, y)

In [ ]:
from scipy.optimize import minimize

def get_best_model(x, y):
  '''
  Returns the model with the parameters that minimize the MSE.

  x: np.array of Floats with shape (n,) of inputs
  y: np.array of Int in {0, 1} with shape (n,) of outputs

  Returns: {'alpha_hat':Float, 'beta_hat':Float} representing the model.
  '''

  res = minimize(function_to_minimize, x0=[0, 0], args=(x, y))
  parameters = res.x
  alpha_hat, beta_hat = parameters[0], parameters[1]
  best_model = dict(alpha_hat=alpha_hat, beta_hat=beta_hat)

  return best_model

In [ ]:
best_model = get_best_model(X_train.flatten(), predictions_df['HeartDisease'])
best_model

{'alpha_hat': 5.516771166306751, 'beta_hat': -0.038253385518181665}

In [ ]:
predictions_df['Our Model Heart Disease Probabilities'] = get_probabilities(best_model, X_train.flatten())
predictions_df

,MaxHR,HeartDisease,Sk Learn Probability,SK Learn Prediction,Our Model Heart Disease Probabilities
0,172,0,0.260939,0,0.256773
1,156,1,0.389737,0,0.389181
2,98,0,0.845538,1,0.854201
3,108,1,0.790775,1,0.799858
4,122,0,0.692325,1,0.700537
...,...,...,...,...,...
913,132,1,0.608400,1,0.614750
914,141,1,0.526777,1,0.530722
915,115,1,0.744655,1,0.753549
916,174,1,0.246907,0,0.242446


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['HeartDisease'],
                         mode='markers', name='Observed', marker=dict(color='orange')))

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['Sk Learn Probability'],
                         mode='markers', name='SK Learn Probabilities', marker=dict(color='blue')))

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=predictions_df['Our Model Heart Disease Probabilities'],
                         mode='markers', name='Our Model Heart Disease Probabilities', marker=dict(color='red')))

fig.add_trace(go.Scatter(x=predictions_df['MaxHR'], y=[.5]*len(predictions_df), 
                         name='Threshold', marker={'color':'purple'}))

fig.update_layout(
    title="Heart Disease vs Max Heart Rate",
    xaxis_title="Max Heart Rate",
    yaxis_title="Heart Disease"
)

fig.show()

In [ ]:
accuracy_score(predictions_df['Our Model Heart Disease Probabilities']>.5, y_train)

0.6721132897603486

In [ ]:
heart.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
train = heart[['Age','RestingBP',	'Cholesterol','MaxHR', 'HeartDisease']].to_numpy()
X_train, y_train = train[:, :-1], train[:, -1]
X_train.shape, y_train.shape

((918, 4), (918,))

In [ ]:
lr = LogisticRegression().fit(X_train, y_train)

accuracy_score(lr.predict(X_train), y_train)

0.7015250544662309